In [1]:
import torch
import torchani
import ase
from ase.optimize import BFGS
from ase.visualize import view

/home/nick/TorchANI_sandbox/torchani/infer.py:16: UserWarning: mnp not installed
  warnings.warn("mnp not installed")
/home/nick/TorchANI_sandbox/torchani/aev/aev_computer.py:24: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")


In [2]:
ani2x = torchani.models.ANI2x()

/home/nick/TorchANI_sandbox/torchani/models.py:103: UserWarning: The default is now to accept atomic numbers as indexes, do not set periodic_table_index=True. if you need to accept raw indices set periodic_table_index=False
  warnings.warn("The default is now to accept atomic numbers as indexes,"


In [3]:
device = torch.device('cpu')
o_min_ch4 = (torch.tensor([[6, 1, 1, 1, 1]]),
           torch.tensor([[[-3.2947064558, 0.6540086535, 0.4313881687],
                          [-2.2029514828, 0.6763552943, 0.4373590881],
                          [-3.6556797933, 0.7756014079, -0.5920264340],
                          [-3.6788104313, 1.4664338298, 1.0517860346],
                          [-3.6413841972, -0.3023558975, 0.8284340142]]]))
min_ch4 = [c.to(device) for c in o_min_ch4]
bad_ch4 = (torch.tensor([[6, 1, 1, 1, 1]]), 
           torch.tensor([[[-2.97117, -0.04326,  0.00000],
                          [-1.87917, -0.04326, -0.00000],
                          [-3.33517, -0.04326,  3.01880],
                          [-3.33517, -0.99976, -0.38088],
                          [-3.33517,  0.76485, -0.63791]]]))
bad_ch4 = [c.to(device) for c in bad_ch4]
really_bad_ch4 = (torch.tensor([[6, 1, 1, 1, 1]]),
                  torch.tensor([[[-3.03512,  1.26157,  0.26327],
                                 [-2.69779,  0.12977,  0.00000],
                                 [-4.12445, -0.59901,  0.69754],
                                 [-4.12445, -0.10992, -0.97991],
                                 [-4.12445,  1.09825,  0.28237]]]))
really_bad_ch4 = [c.to(device) for c in really_bad_ch4]

In [4]:
min_ch4 = o_min_ch4

In [5]:
calculator = ani2x.ase()

In [6]:
ase_input = ase.Atoms(numbers=min_ch4[0][0], positions=min_ch4[1][0])
ase_input.calc = calculator
print(o_min_ch4)

opt = BFGS(ase_input)                 
opt.run(fmax=0.0001)                   
print(ase_input.get_positions())      

new_coords = torch.tensor(ase_input.get_positions())[None]
min_ch4 = (min_ch4[0], new_coords.float())
print(min_ch4)

(tensor([[6, 1, 1, 1, 1]]), tensor([[[-3.2947,  0.6540,  0.4314],
         [-2.2030,  0.6764,  0.4374],
         [-3.6557,  0.7756, -0.5920],
         [-3.6788,  1.4664,  1.0518],
         [-3.6414, -0.3024,  0.8284]]]))
      Step     Time          Energy         fmax
BFGS:    0 08:45:28    -1102.045251        0.0168
BFGS:    1 08:45:28    -1102.045263        0.0083
BFGS:    2 08:45:28    -1102.045268        0.0000
[[-3.29470649  0.6540086   0.43138822]
 [-2.20247775  0.67636501  0.4373617 ]
 [-3.65583629  0.77565413 -0.59247042]
 [-3.67897694  1.46678632  1.0520552 ]
 [-3.64153448 -0.30277075  0.82860621]]
(tensor([[6, 1, 1, 1, 1]]), tensor([[[-3.2947,  0.6540,  0.4314],
         [-2.2025,  0.6764,  0.4374],
         [-3.6558,  0.7757, -0.5925],
         [-3.6790,  1.4668,  1.0521],
         [-3.6415, -0.3028,  0.8286]]]))


/home/nick/TorchANI_sandbox/torchani/ase.py:51: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352660876/work/torch/csrc/utils/tensor_new.cpp:201.)
  cell = torch.tensor(self.atoms.get_cell(complete=True),


In [7]:
print(ani2x.atomic_qbcs(bad_ch4).energies)
print(ani2x.atomic_qbcs(bad_ch4).ae_stdev)

tensor([[[ 0.1986,  0.0043,  0.1656,  0.0044,  0.0045]],

        [[ 0.2141, -0.0052,  0.1799, -0.0052, -0.0052]],

        [[ 0.2178, -0.0025,  0.1989, -0.0027, -0.0029]],

        [[ 0.1923,  0.0044,  0.1861,  0.0043,  0.0041]],

        [[ 0.1948,  0.0077,  0.1815,  0.0081,  0.0083]],

        [[ 0.1970,  0.0021,  0.1695,  0.0022,  0.0021]],

        [[ 0.2092,  0.0029,  0.1958,  0.0029,  0.0027]],

        [[ 0.1931,  0.0031,  0.1997,  0.0033,  0.0034]]])
tensor([[0.0101, 0.0041, 0.0130, 0.0042, 0.0043]])


In [8]:
print(ani2x.atomic_qbcs(min_ch4).energies)
print(ani2x.atomic_qbcs(min_ch4).ae_stdev)

tensor([[[-0.0071, -0.0029, -0.0029, -0.0029, -0.0029]],

        [[ 0.0240, -0.0107, -0.0107, -0.0107, -0.0107]],

        [[ 0.0026, -0.0053, -0.0053, -0.0053, -0.0053]],

        [[-0.0131, -0.0014, -0.0014, -0.0014, -0.0014]],

        [[-0.0125, -0.0015, -0.0015, -0.0015, -0.0015]],

        [[ 0.0108, -0.0074, -0.0074, -0.0074, -0.0074]],

        [[-0.0027, -0.0040, -0.0040, -0.0040, -0.0040]],

        [[-0.0065, -0.0029, -0.0029, -0.0029, -0.0029]]])
tensor([[0.0127, 0.0032, 0.0032, 0.0032, 0.0032]])


In [9]:
bad_ch4_view = ase.Atoms(numbers=bad_ch4[0][0], positions=bad_ch4[1][0])
view(bad_ch4_view, viewer='ngl')

In [12]:
min_ch4_view = ase.Atoms(numbers=min_ch4[0][0], positions=min_ch4[1][0])
view(min_ch4_view, viewer='ngl')

In [15]:
import rdkit
from rdkit import Chem

In [18]:
Chem.MolFromXYZBlock(bad_ch4[1])
#Chem.MolFragmentToSmarts(bad_ch4)

TypeError: No registered converter was able to produce a C++ rvalue of type std::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type Tensor